*Автор: Алексей Полтавец*<br />

# <center> Introduction
Туториал вдохновлен [kernels](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline) и являеться его воплощением на примере [QR кодов](https://www.kaggle.com/c/receipt-categorisation)
Далее будем расматривать что такое NBSVM (Naive Bayes - Support Vector Machine) и как его применяют для линейных моделей.

NBSVM представлена Sida Wang и Chris Manning в работе: [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf).

Naive Bayes (NB) и Support Vector Machine (SVM) модель, которую используют для улучшения базового линейного метода для категоризации.

### <center>Немного теории из статьи:
 
Базовый линейный метод имеет математическую формулировку:
    $y = w_0 + \sum_{i=1}^m w_i x_i$
Задача линейной модели найти коэффициенты $w_i$ при $x_i$.

Naive Bayes - Support Vector Machine выполняет вспомагательную функцию (что указано в названии). Суть метода преобразовать коэффициенты $x_i$ так, что бы они были больше для целевой категории и поменьше для других (метод основан на бинарном представлении класса как One vs All). Для этого предлагаеться $x_i$ домножить на коэффициент важности переменной (в тексте для токена). Важность токена выводиться из формулы Байеса:

Формула Байеса имеет вид:

$$\large\begin{array}
\mathcal{P}\left(A\mid B\right) &=& \frac{P\left(B\mid A\right)P\left(A\right)}{P\left(B\right)}
\end{array}$$

Распишем уравнение для задачи вычисления вероятности отнесения строки к классу $c=1$:

$$\large\begin{array}
\mathcal{P}\left(c=1\mid d\right) &=& \frac{P\left(d\mid c=1\right)P\left(c=1\right)}{P\left(d\right)}
\end{array}$$
Разделим все на вероятность отнесения к классу $c=0$:

$$\large\begin{array}
\mathcal\frac{P\left(c=1\mid d\right)}{P\left(c=0\mid d\right)} &=& \frac{P\left(d\mid c=1\right)P\left(c=1\right)}{P\left(d\right)}\frac{P\left(d\right)}{P\left(d\mid c=0\right)P\left(c=0\right)}
\end{array}$$

Если полученое больше 1 - скорей всего $c=1$, если меньше - $c=0$. 

Взяв логорифм выражения получим:

$$\large\begin{array}
\mathcal{r}&=& \log\left(\frac{p/\left\|p\right\|_1}{q/\left\| q\right\|_1}\right)
\end{array}$$


Где: $\begin{array}\mathcal{p}&=& \alpha + \sum_{i : y^\left(i\right) = 1} f^\left(i\right)\end{array}$ и
$\begin{array}\mathcal{q}&=& \alpha + \sum_{i : y^\left(i\right) = 0} f^\left(i\right)\end{array}$, $\alpha$ - коэффициент сглаживания (выбираеться исходя из задачи). 

$p$ то как часто данный $x_i$ встречаеться при $y = 1$, $q$ - при $y = 0$.

Перейдем к воплощению метода в коде:

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

## Read data

In [ ]:
train = pd.read_csv("train.csv.gz")
train.fillna("", inplace=True)

In [ ]:
vectorizer = CountVectorizer()
labeler = LabelEncoder()
lb = LabelBinarizer()

In [ ]:
train.head(3)

In [ ]:
X = vectorizer.fit_transform(train.name.values)

In [ ]:
y = labeler.fit_transform(train.category)


Разбиваем обучающую выборку на части. Для демонстрации NB-SVM будем брать только первый fold.


In [ ]:
gkf = list(GroupKFold(n_splits=3).split(X, y, train.check_id.values))

Создадим матрицу в бинарном представлении категорий:

In [ ]:
labels = lb.fit_transform(y)

In [ ]:
logist = LogisticRegression()
mnb = MultinomialNB()

Быстрая проверка какие результаты дают LogisticRegression и MultinomialNB.

In [ ]:
%%time
score = cross_val_score(logist, X, y, scoring="neg_log_loss", cv=gkf, n_jobs=-1)
print("%.3f +- %.4f" % (-np.mean(score), np.std(score)))

In [ ]:
print("Результаты для первого фолда методом LogisticRegression = %.3f" % (-score[0]))

In [ ]:
%%time
score = cross_val_score(mnb, X, y, scoring="neg_log_loss", cv=gkf, n_jobs=-1)
print("%.3f +- %.4f" % (-np.mean(score), np.std(score)))

In [ ]:
print("Результаты для первого фолда методом MultinomialNB = %.3f" % (-score[0]))

Как видим оба метода примерно одинаковы по результатам (MultinomialNB слегка лучше).

Разделим train на X_train и X_valid, взяв за разбиение первый фолд созданый GroupKFold


#### Use 1st slit for example NB - SVM

In [ ]:
X_train, X_valid, y_train, y_valid = (
    X[gkf[0][0]],
    X[gkf[0][1]],
    labels[gkf[0][0], :],
    labels[gkf[0][1], :],
)

In [ ]:
# Разбивка на 3 части : у нас 0,66 train part и 0,33 valid part
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

Кратко о MultinomialNB. Модель создает матрицу в которой указываеться, как часто каждая из характеристик (у нас тут tokens из слов) встречаеться при конкретном классе. Параметр сглаживания $\alpha$ являеться гиперпараметром для модели. В формуле ratio у нас под логорифмом дробь, по правилам логарифмирования это разница логарифмов.
$$\begin{array}
\mathcal{r}&=& \log\left(\frac{p/\left\|p\right\|_1}{q/\left\| q\right\|_1}\right)
&=& \log\left(p/\left\|p\right\|_1\right) - \log\left(q/\left\|q\right\|_1\right)
\end{array}$$
У MultinomialNB есть метод который вернет нам эти слогаемые: feature_log_prob_$[1]$ и feature_log_prob_$[0]$. Остаеться применить полученые коэффициенты к data.



In [ ]:
preds = []
for idx, category in enumerate(labeler.classes_):
    # print (' Prediction for %s' % category)
    mnb.fit(X_train, y_train[:, idx])
    ratio = mnb.feature_log_prob_[1] - mnb.feature_log_prob_[0]
    logist.fit(X_train.multiply(ratio), y_train[:, idx])
    preds.append(logist.predict_proba(X_valid.multiply(ratio))[:, 1])

In [ ]:
print(
    "Score after applying NB-SVM on data = %.3f" % log_loss(y_valid, np.array(preds).T)
)

Результат NB-SVM лечше отдельно каждого их методов 0,799 и 0,744. После применения имеем 0,650.